In [1]:
import pandas as pd
import pickle
from tqdm import tqdm
import torch
import networkx as nx
from GraphDataset import GraphDataset
from torch_geometric.utils.convert import to_networkx, from_networkx

ModuleNotFoundError: No module named 'pandas'

In [2]:
df = pd.read_pickle('/global/scratch/tlundy/NFT_Research/nft_research/Dino/archive/graph_images_dataframe.pkl')
df

,Label,Features,Collection,NFT_num,AverageFeatureVector
0,0,"[0.006129133980721235, 0.08498869091272354, -0...",-glowa-,1,"[-0.007021066558081657, 0.034550966462120414, ..."
1,0,"[0.04774976521730423, 0.05362828075885773, 0.0...",-glowa-,10017,"[-0.007021066558081657, 0.034550966462120414, ..."
2,0,"[0.10529650747776031, 0.07033471018075943, 0.0...",-glowa-,10018,"[-0.007021066558081657, 0.034550966462120414, ..."
3,0,"[-0.02233046293258667, 0.01190363708883524, -0...",-glowa-,10019,"[-0.007021066558081657, 0.034550966462120414, ..."
4,0,"[0.04859286919236183, 0.054554957896471024, 0....",-glowa-,10024,"[-0.007021066558081657, 0.034550966462120414, ..."
...,...,...,...,...,...
456139,10976,"[-0.033141832798719406, -0.0350814163684845, -...",zzz-zzz-by-hashlips,75,"[-0.033141832798719406, -0.0350814163684845, -..."
456140,10976,"[-0.033141832798719406, -0.0350814163684845, -...",zzz-zzz-by-hashlips,77,"[-0.033141832798719406, -0.0350814163684845, -..."
456141,10976,"[-0.033141832798719406, -0.0350814163684845, -...",zzz-zzz-by-hashlips,78,"[-0.033141832798719406, -0.0350814163684845, -..."
456142,10976,"[-0.033141832798719406, -0.0350814163684845, -...",zzz-zzz-by-hashlips,79,"[-0.033141832798719406, -0.0350814163684845, -..."


In [3]:
df = df.drop_duplicates('Label')
df = df.drop(columns=['Features', 'NFT_num'])
df

,Label,Collection,AverageFeatureVector
0,0,-glowa-,"[-0.007021066558081657, 0.034550966462120414, ..."
12,1,-jubilees-multiverse,"[-0.0728024194156751, 0.02133718698984012, -0...."
62,2,-phunks,"[0.00896344568929635, 0.032312632966786624, -0..."
112,3,-subtraction-,"[-0.001622253995574283, 0.008428905807086266, ..."
162,4,0-project,"[-0.016895037296926603, 0.030957878581248224, ..."
...,...,...,...
455935,10972,zunkpets,"[-0.043920986556913705, 0.004254778374452144, ..."
455985,10973,zunkz,"[-0.002928928262554109, 0.01807572929188609, -..."
456035,10974,zuphioh-editions,"[-0.07176712848660019, -0.012588592763576243, ..."
456044,10975,zuttomamoru,"[-0.1024509184807539, 0.018767129573971034, -0..."


In [4]:
feature_vector_dict = dict(zip(df.Label, df.AverageFeatureVector))

In [5]:
feature_vector_dict[10976]

array([-3.31418328e-02, -3.50814164e-02, -4.13184911e-02,  1.38853341e-02,
       -3.02126724e-02, -1.83182042e-02, -8.37744102e-02, -1.21998377e-02,
        2.50642244e-02,  1.79000143e-02,  6.16281331e-02,  6.24127919e-03,
       -2.40286849e-02,  1.05421315e-03,  7.32735172e-02,  1.50881177e-02,
       -4.18015011e-03, -2.42279470e-02,  1.61784627e-02, -7.56789967e-02,
        1.16689457e-02, -6.66706488e-02, -1.23137869e-02, -7.79725164e-02,
        8.08669080e-04, -5.28362542e-02,  4.04609852e-02,  3.10569489e-03,
        4.00739387e-02, -6.31217659e-02, -6.77243248e-02, -2.17949655e-02,
        6.19085468e-02, -4.25417498e-02,  1.92671847e-02,  5.87103292e-02,
        6.16602115e-02,  5.76729476e-02, -9.26565751e-02, -3.37369964e-02,
        9.04059783e-02, -3.04079298e-02, -5.86332232e-02,  3.32158594e-03,
       -8.78510065e-03, -3.61300260e-02, -1.66696787e-04,  2.00314093e-02,
        1.21715339e-02,  6.89247996e-02,  9.30368453e-02,  2.00049449e-02,
       -1.21828273e-01, -

In [6]:
dataset = GraphDataset('dataset_stor/graph_dataset_4_processed/graphs/')

In [8]:
for i, graph in enumerate(dataset):
    graph = to_networkx(graph, node_attrs=['collection', 'label', 'x', 'y'])
    collection_nodes = [node for node in graph if graph.nodes[node]['collection']]
    for node in tqdm(collection_nodes):
        try:
            graph.nodes[node]['x'] = feature_vector_dict[graph.nodes[node]['label']]
        except KeyError:
            graph.remove_node(node)
            graph.remove_nodes_from(list(nx.isolates(graph)))
    

    pyg_graph = from_networkx(graph)
    torch.save(pyg_graph, f'dataset_stor/graph_dataset_4_processed/graphs_w_feature_vectors/graph_{i}.pt')
    # break

100%|██████████| 979/979 [00:00<00:00, 127788.37it/s]


KeyboardInterrupt: 

In [3]:
import pickle

with open('dataset_stor/graph_dataset_4/label_map.pkl', 'rb') as f:
    vector  = pickle.load(f)
    print(vector)

{'lny-bobu-card': 1, 'pixters-nfts': 2, 'nakacolaks': 3, 'openpalette': 4, 'yaypepe-club': 5, 'hashdemons': 6, 'flipmap': 7, 'squishy-apes-nft': 8, 'cryptoadz-by-gremplin': 9, 'nekavolve': 10, 'konneecheewahs': 11, 'ultiverse-electric-sheep': 12, 'braintoadz': 13, 'aq-editions': 14, 'reli3fukr-drop2': 15, 'undefined1231231': 16, 'oriental-deep-eyes': 17, 'mint-fun-turns-one-1': 18, 'loyalty-program-4': 19, 'sandbox': 20, 'moonordietrying': 21, 'theloomiheads': 22, 'cool-cats-nft': 23, 'satoshis-legions-presale-tokens': 24, 'sweets-on-chain-candies': 25, 'ringerpunks': 26, 'matt-gondek': 27, 'unitedplanets': 28, 'nonconformistducks': 29, 'animalsnotfood': 30, 'decentraland': 31, 'w0rdl3': 32, 'thepotatoz': 33, 'thecaptainz': 34, 'pajiro-pass-genesis': 35, 'the-nfg': 36, 'emblem-vault': 37, 'vlvtdrops': 38, 'girlies-nft': 39, 'dot-pigeon': 40, 'oblivious-ostriches': 41, 'umucos-art': 42, 'sinsoneth': 43, 'ghosties-nfts': 44, 'apocalyptic-apes': 45, 'king-of-midtown': 46, 'splattersgen1':